# Model 1: Implementation

In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# Residual for single point
def r(A, c, zi, wi):
    v = zi - c
    x = v.T @ A @ v - 1
    return np.maximum(wi * x, 0)

# Residual vector
def R(A, c, Z, W):
    R = np.zeros_like(W)
    for i in range(R.size):
        R[i] = r(A, c, Z[i], W[i])
    return R


# Objective function
def f(x, Z, W):
    m, n = Z.shape
    A = np.zeros((n, n))
    c = np.zeros(n)
    update_A(A, x)
    update_c(c, x)
    return np.sum(R(A, c, Z, W)**2)


# Gradient of objective function. Returns (n+k) vector
def df(x, Z, W):
    m, n = Z.shape
    A = np.zeros((n, n))
    c = np.zeros(n)
    update_A(A, x)
    update_c(c, x)
    print(R(A, c, Z, W))
    print(jacobian_R(A, c, Z))
    return jacobian_R(A, c, Z).T @ R(A, c, Z, W)

In [3]:
# Insert coefficients in (nxn) matrix A, given (m+k) vector x
def update_A(A, x):
    n = A.shape[0]
    k = n*(n+1)//2
    a = x[:k]
    
    end = 0
    for j in range(n):
        start = end
        end = start + n-j
        A[j, j:] = x[start:end]
        A[j+1:,j] = A[j, j+1:]
    
    return A

# Insert coefficients in (n) centering vector given (n+k) vector x
def update_c(c, x):
    n = c.size
    c = x[-n:]
    return c

# Get the first (k) coefficients of (n+k) vector x, given an (nxn) matrix A
def extract_a(A):
    n = A.shape[0]
    k = n*(n+1)//2
    a = np.zeros(k)
    end = 0
    
    for j in range(n):
        start = end
        end = start + n-j
        a[start:end] = A[j, j:]
        
    return a

In [4]:
# return partial derivatives of residual i, wrt. the matrix coefficients
def r_delA(A, c, zi):
    v = zi - c
    rdA = 2 * (v * (v * np.ones_like(A)).T)
    rdA = rdA - 1/2 * np.identity(c.size) * rdA
    return extract_a(rdA)

# return partial derivatives of residual i, wrt. centering vector coefficients
def r_delc(A, c, zi):
    v = zi - c
    return - 2 * (v.T @ A)

# Return vector of all partial derivatives of residual i
def del_r(A, c, zi):
    v = zi - c
    n = c.size
    k = n*(n+1)//2
    dr = np.zeros(k+n)
    dr[:k] = r_delA(A, c, zi)
    dr[k:] = r_delc(A, c, zi)
    return dr

#Jacobian of a all residual points. 
def jacobian_R(A, c, Z):
    m, n = Z.shape
    k = n*(n+1)//2
    R = np.zeros((m, n+k))
    for i in range(m):
        R[i] = del_r(A, c, Z[i])
    return R

In [5]:
Z = np.array((2, 3, 2, 4, 4, 5)).reshape(3, 2)
W = np.ones(3)
x = np.array((1, 0, 0, 4, 1, 0))
f(x, Z, W)

243.0

In [6]:
df(x, Z, W)

[  3.   3.  15.]
[[  4.  12.   9.  -4.  -0.]
 [  4.  16.  16.  -4.  -0.]
 [ 16.  40.  25.  -8.  -0.]]


array([ 264.,  684.,  450., -144.,    0.])